***GENERATED CODE FOR airpass1409 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import pyspark
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AIRPassengerTrainData[Month]', 'transformation_label': 'String Indexer'}], 'feature': 'AIRPassengerTrainData[Month]', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '144', 'mean': '', 'stddev': '', 'min': 'April', 'max': 'September', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AIRPassengerTrainData[Month]'}, {'feature_label': 'AIRPassengerTrainData[Month]', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'AIRPassengerTrainData[Month]')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
from fbprophet import Prophet
import pandas as pd


def driverProphet(df):
    df.columns = ['ds', 'y']
    df['ds'] = df['ds'].astype(str).str[:-6]
    df['ds'] = pd.to_datetime(df['ds'])
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    m.plot(forecast)
    return m


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://backendmaster:7077').getOrCreate()
#%run airpass1409Hooks.ipynb
try:
	#sourcePreExecutionHook()

	airpasshdfs = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/extra/tempFile1694673844.csv', 'filename': 'tempFile1694673844.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(airpasshdfs)

except Exception as ex: 
	logging.error(ex)
spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run airpass1409Hooks.ipynb
try:
	#transformationPreExecutionHook()

	airpassfeatureforecast = TransformationTimeSeriesForecastingMain.run(airpasshdfs, json.dumps( {"FE": {"dataPercentage": 10, "featureList": [{"transformationsData": [{}], "feature": "AIRPassengerTrainData[Date]", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AIRPassengerTrainData[Date]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AIRPassengerTrainData[#Passengers]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "144", "mean": "280.3", "stddev": "119.97", "min": "104", "max": "622", "missing": "0"}, "updatedLabel": "AIRPassengerTrainData[#Passengers]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AIRPassengerTrainData[Year]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "144", "mean": "1954.5", "stddev": "3.46", "min": "1949", "max": "1960", "missing": "0"}, "updatedLabel": "AIRPassengerTrainData[Year]"}, {"transformationsData": [{"feature_label": "AIRPassengerTrainData[Month]", "transformation_label": "String Indexer"}], "feature": "AIRPassengerTrainData[Month]", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "144", "mean": "", "stddev": "", "min": "April", "max": "September", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AIRPassengerTrainData[Month]"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AIRPassengerTrainData[MonthNumber]", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "144", "mean": "6.5", "stddev": "3.46", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "AIRPassengerTrainData[MonthNumber]"}, {"feature": "AIRPassengerTrainData[Month]_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "144", "mean": "5.5", "stddev": "3.46", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "AIRPassengerTrainData[Month]_stringindexer_transform"}], "features": {"timecolumn": "AIRPassengerTrainData[Date]", "tocompare": "AIRPassengerTrainData[#Passengers]"}, "functionList": [{"function": "rollingmean"}], "originalfile": "/FileStore/platform/extra/tempFile1694673844.csv", "statFunction": {"function": "Original", "parameter": ""}}}))

	#transformationPostExecutionHook(airpassfeatureforecast)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run airpass1409Hooks.ipynb
try:
	#mlPreExecutionHook()

	model = driverProphet(airpassfeatureforecast)

	#mlPostExecutionHook(model)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
try:
        future = model.make_future_dataframe(periods=30,freq='MS',include_history=False)
        predictedDataFrame = model.predict(future)
        display(model.plot(predictedDataFrame))
        display(model.plot_components(predictedDataFrame))
        display(predictedDataFrame)
except Exception as ex:
    logging.error(ex)

